In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch

print("Numpy:", np.__version__)
import matplotlib
print("Matplotlib:", matplotlib.__version__)

print("OpenCV:", cv2.__version__)
print("PyTorch:", torch.__version__)
print("GPU disponível:", torch.cuda.is_available())


Numpy: 2.2.3
Matplotlib: 3.5.3
OpenCV: 4.11.0
PyTorch: 2.6.0+cu124
GPU disponível: False


In [2]:
import os


DATASET_DIR = "../datasets"
IMAGE_DIR_TRAIN = os.path.join(DATASET_DIR, "image", "train")
MASK_DIR_TRAIN = os.path.join(DATASET_DIR, "mask", "train")
IMAGE_DIR_VAL = os.path.join(DATASET_DIR, "image", "validation")
MASK_DIR_VAL = os.path.join(DATASET_DIR, "mask", "validation")
#SUBINDO UM NIVEL NOS CAMINHOOS

In [3]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
train_images = sorted(os.listdir(IMAGE_DIR_TRAIN))
train_masks = sorted(os.listdir(MASK_DIR_TRAIN))
val_images = sorted(os.listdir(IMAGE_DIR_VAL))
val_masks = sorted(os.listdir(MASK_DIR_VAL))

# Função para carregar uma imagem e sua máscara correspondente
def load_image_mask(image_path, mask_path):
    image = cv2.imread(image_path)  # Lê a imagem
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Converte BGR para RGB
    image = cv2.resize(image, (256, 256))  # Redimensiona

    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # Lê a máscara em escala de cinza
    mask = cv2.resize(mask, (256, 256))  # Redimensiona
    mask = (mask > 0).astype(np.uint8)  # Binariza (0 = fundo, 1 = lesão)

    return image, mask

# Exibir algumas amostras
def plot_samples(image_list, mask_list, dataset_type="Treino"):
    fig, axes = plt.subplots(len(image_list), 2, figsize=(10, 5 * len(image_list)))
    for i, (img_name, mask_name) in enumerate(zip(image_list, mask_list)):
        img_path = os.path.join(IMAGE_DIR_TRAIN if dataset_type == "Treino" else IMAGE_DIR_VAL, img_name)
        mask_path = os.path.join(MASK_DIR_TRAIN if dataset_type == "Treino" else MASK_DIR_VAL, mask_name)

        image, mask = load_image_mask(img_path, mask_path)

        axes[i, 0].imshow(image)
        axes[i, 0].set_title(f"Imagem: {img_name}")
        axes[i, 0].axis("off")

        axes[i, 1].imshow(mask, cmap="gray")
        axes[i, 1].set_title(f"Máscara: {mask_name}")
        axes[i, 1].axis("off")

    plt.tight_layout()
    plt.show()

# Exibir 5 amostras do conjunto de treino
print("🔍 Visualizando imagens e máscaras do conjunto de TREINO:")
plot_samples(train_images[:5], train_masks[:5], "Treino")

# Exibir 5 amostras do conjunto de validação
print("🔍 Visualizando imagens e máscaras do conjunto de VALIDAÇÃO:")
plot_samples(val_images[:5], val_masks[:5], "Validação")
#TESTANDO DATASET E CORRESPONDENCIAS

🔍 Visualizando imagens e máscaras do conjunto de TREINO:


AttributeError: `np.Inf` was removed in the NumPy 2.0 release. Use `np.inf` instead.

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f1f72cd0400> (for post_execute), with arguments args (),kwargs {}:


AttributeError: `np.Inf` was removed in the NumPy 2.0 release. Use `np.inf` instead.

AttributeError: `np.Inf` was removed in the NumPy 2.0 release. Use `np.inf` instead.

<Figure size 1000x2500 with 10 Axes>

In [3]:
import os
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

# Definição do dataset customizado
class ISICDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.image_list = sorted(os.listdir(image_dir))
        self.mask_list = sorted(os.listdir(mask_dir))
        self.transform = transform

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        # Garante que o índice não vai além do tamanho do dataset
        idx = idx % len(self.image_list)

        # Caminhos das imagens e máscaras
        img_path = os.path.join(self.image_dir, self.image_list[idx])
        mask_name = self.image_list[idx].replace('.jpg', '_segmentation.png')
        mask_path = os.path.join(self.mask_dir, mask_name)

        # Verificação de Existência
        if not os.path.exists(img_path) or not os.path.exists(mask_path):
            print(f"⚠️ Arquivo não encontrado: {img_path} ou {mask_path}")
            return self.__getitem__((idx + 1) % len(self.image_list))

        # Carregar Imagem e Máscara
        image = cv2.imread(img_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Verificação de Corrupção
        if image is None or mask is None:
            print(f"⚠️ Imagem ou Máscara corrompida em: {img_path} ou {mask_path}")
            return self.__getitem__((idx + 1) % len(self.image_list))

        # Conversão e Redimensionamento
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (256, 256))
        mask = cv2.resize(mask, (256, 256))

        # Normalização
        image = image / 255.0
        mask = (mask > 0).astype(np.float32)  # Converte a máscara para binário (0 ou 1)

        # Conversão para Tensor
        image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)
        mask = torch.tensor(mask, dtype=torch.float32).unsqueeze(0)

        return image, mask


# Criar dataset e DataLoader
train_dataset = ISICDataset("../datasets/image/train", "../datasets/mask/train")
val_dataset = ISICDataset("../datasets/image/validation", "../datasets/mask/validation")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, drop_last=True)

# Teste: Exibir um batch do DataLoader
batch = next(iter(train_loader))
images, masks = batch

print(f"📊 Formato do batch de imagens: {images.shape}")  # Esperado: [8, 3, 256, 256]
print(f"📊 Formato do batch de máscaras: {masks.shape}")  # Esperado: [8, 1, 256, 256]


📊 Formato do batch de imagens: torch.Size([8, 3, 256, 256])
📊 Formato do batch de máscaras: torch.Size([8, 1, 256, 256])


In [4]:
import torch.nn as nn
import torch.nn.functional as F

# Bloco de convolução dupla (Conv -> ReLU -> Conv -> ReLU)
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

# Arquitetura da U-Net
class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super(UNet, self).__init__()

        # Downsampling (Encoder)
        self.conv1 = DoubleConv(in_channels, 64)
        self.conv2 = DoubleConv(64, 128)
        self.conv3 = DoubleConv(128, 256)
        self.conv4 = DoubleConv(256, 512)
        self.conv5 = DoubleConv(512, 1024)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Upsampling (Decoder)
        self.up6 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.conv6 = DoubleConv(1024, 512)

        self.up7 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.conv7 = DoubleConv(512, 256)

        self.up8 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.conv8 = DoubleConv(256, 128)

        self.up9 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.conv9 = DoubleConv(128, 64)

        # Camada final (Saída)
        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        # Encoder
        c1 = self.conv1(x)
        p1 = self.pool(c1)

        c2 = self.conv2(p1)
        p2 = self.pool(c2)

        c3 = self.conv3(p2)
        p3 = self.pool(c3)

        c4 = self.conv4(p3)
        p4 = self.pool(c4)

        c5 = self.conv5(p4)

        # Decoder
        up6 = self.up6(c5)
        merge6 = torch.cat([up6, c4], dim=1)
        c6 = self.conv6(merge6)

        up7 = self.up7(c6)
        merge7 = torch.cat([up7, c3], dim=1)
        c7 = self.conv7(merge7)

        up8 = self.up8(c7)
        merge8 = torch.cat([up8, c2], dim=1)
        c8 = self.conv8(merge8)

        up9 = self.up9(c8)
        merge9 = torch.cat([up9, c1], dim=1)
        c9 = self.conv9(merge9)

        output = self.final_conv(c9)
        return output

# Teste do modelo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet().to(device)

# Teste com um batch do DataLoader
sample_input = torch.randn(8, 3, 256, 256).to(device)  # Batch fake para teste
output = model(sample_input)

print(f"Formato da saída da U-Net: {output.shape}")  # Esperado: [8, 1, 256, 256]


Formato da saída da U-Net: torch.Size([8, 1, 256, 256])


In [5]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import time

# Configuração do treinamento
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet().to(device)

# Função de perda (Binary Cross Entropy com Dice Loss)
def dice_loss(pred, target, smooth=1e-5):
    pred = torch.sigmoid(pred)  # Converte logits para probabilidade
    intersection = (pred * target).sum()
    return 1 - ((2. * intersection + smooth) / (pred.sum() + target.sum() + smooth))

criterion = lambda pred, target: 0.5 * F.binary_cross_entropy_with_logits(pred, target) + 0.5 * dice_loss(pred, target)

# Otimizador
optimizer = optim.Adam(model.parameters(), lr=0.0001)


# Funções para métricas
def iou(pred, target, threshold=0.5):
    pred = (torch.sigmoid(pred) > threshold).float()
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum() - intersection
    return (intersection + 1e-5) / (union + 1e-5)

def dice_coefficient(pred, target, threshold=0.5):
    pred = (torch.sigmoid(pred) > threshold).float()
    intersection = (pred * target).sum()
    return (2. * intersection + 1e-5) / (pred.sum() + target.sum() + 1e-5)

def precision(pred, target, threshold=0.5):
    pred = (torch.sigmoid(pred) > threshold).float()
    true_positive = (pred * target).sum()
    predicted_positive = pred.sum()
    return (true_positive + 1e-5) / (predicted_positive + 1e-5)

def recall(pred, target, threshold=0.5):
    pred = (torch.sigmoid(pred) > threshold).float()
    true_positive = (pred * target).sum()
    actual_positive = target.sum()
    return (true_positive + 1e-5) / (actual_positive + 1e-5)

def f1_score(pred, target, threshold=0.5):
    p = precision(pred, target, threshold)
    r = recall(pred, target, threshold)
    return 2 * (p * r) / (p + r + 1e-5)

# Função para treinar o modelo com métricas
def train_model(model, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        start_time = time.time()

        # Métricas de treino
        train_iou, train_dice, train_precision, train_recall, train_f1 = 0, 0, 0, 0, 0
        
        for images, masks in train_loader:
            images, masks = images.to(device), masks.to(device)

            # Forward
            outputs = model(images)
            loss = criterion(outputs, masks)

            # Backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

            # Calcular métricas para o batch
            train_iou += iou(outputs, masks).item()
            train_dice += dice_coefficient(outputs, masks).item()
            train_precision += precision(outputs, masks).item()
            train_recall += recall(outputs, masks).item()
            train_f1 += f1_score(outputs, masks).item()

        # Validação
        model.eval()
        val_loss = 0
        val_iou, val_dice, val_precision, val_recall, val_f1 = 0, 0, 0, 0, 0
        
        with torch.no_grad():
            for images, masks in val_loader:
                images, masks = images.to(device), masks.to(device)
                outputs = model(images)
                val_loss += criterion(outputs, masks).item()

                # Calcular métricas para o batch
                val_iou += iou(outputs, masks).item()
                val_dice += dice_coefficient(outputs, masks).item()
                val_precision += precision(outputs, masks).item()
                val_recall += recall(outputs, masks).item()
                val_f1 += f1_score(outputs, masks).item()

        # Exibir progresso
        print(f"📌 Época [{epoch+1}/{epochs}] - Loss Treino: {epoch_loss/len(train_loader):.4f} | Loss Validação: {val_loss/len(val_loader):.4f}")
        print(f"   Métricas de Treino -> IoU: {train_iou/len(train_loader):.4f} | Dice: {train_dice/len(train_loader):.4f} | Prec: {train_precision/len(train_loader):.4f} | Rec: {train_recall/len(train_loader):.4f} | F1: {train_f1/len(train_loader):.4f}")
        print(f"   Métricas de Validação -> IoU: {val_iou/len(val_loader):.4f} | Dice: {val_dice/len(val_loader):.4f} | Prec: {val_precision/len(val_loader):.4f} | Rec: {val_recall/len(val_loader):.4f} | F1: {val_f1/len(val_loader):.4f}")
        print(f"   Tempo da Época: {time.time() - start_time:.2f}s")

# Treinar o modelo com métricas
train_model(model, train_loader, val_loader, epochs=50)


📌 Época [1/50] - Loss Treino: 0.5210 | Loss Validação: 0.4561
   Métricas de Treino -> IoU: 0.2978 | Dice: 0.3953 | Prec: 0.8306 | Rec: 0.3881 | F1: 0.3953
   Métricas de Validação -> IoU: 0.4446 | Dice: 0.5913 | Prec: 0.7725 | Rec: 0.5511 | F1: 0.5913
   Tempo da Época: 1113.63s
📌 Época [2/50] - Loss Treino: 0.4096 | Loss Validação: 0.3655
   Métricas de Treino -> IoU: 0.4967 | Dice: 0.6504 | Prec: 0.7803 | Rec: 0.6048 | F1: 0.6504
   Métricas de Validação -> IoU: 0.5001 | Dice: 0.6562 | Prec: 0.9410 | Rec: 0.5179 | F1: 0.6562
   Tempo da Época: 1108.77s
📌 Época [3/50] - Loss Treino: 0.2975 | Loss Validação: 0.3263
   Métricas de Treino -> IoU: 0.6293 | Dice: 0.7657 | Prec: 0.8326 | Rec: 0.7387 | F1: 0.7657
   Métricas de Validação -> IoU: 0.6188 | Dice: 0.7567 | Prec: 0.8764 | Rec: 0.6842 | F1: 0.7567
   Tempo da Época: 1122.89s
📌 Época [4/50] - Loss Treino: 0.2530 | Loss Validação: 0.2771
   Métricas de Treino -> IoU: 0.6717 | Dice: 0.7989 | Prec: 0.8434 | Rec: 0.7795 | F1: 0.7989
 

In [6]:
print("📊 Tamanho do Dataset")
print(f"Treino - Total de Imagens: {len(train_loader.dataset)}")
print(f"Validação - Total de Imagens: {len(val_loader.dataset)}")

print("\n📊 Tamanho do DataLoader")
print(f"Treino - Total de Batches: {len(train_loader)}")
print(f"Validação - Total de Batches: {len(val_loader)}")


📊 Tamanho do Dataset
Treino - Total de Imagens: 1474
Validação - Total de Imagens: 101

📊 Tamanho do DataLoader
Treino - Total de Batches: 184
Validação - Total de Batches: 12
